In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
from pyha_analyzer import PyhaTrainer, PyhaTrainingArguments, extractors

# Loading in Data

In [3]:
# Sorry TQ, I don't have these files mounted yet...
# peru132_extr = extractors.Peru132Extractor()
# peru_132_ads = peru132_extr("/data/XC_wav")

In [4]:
# peru_132_ads = peru_132_ads.get_provenance()

In [27]:
coralreef_extractor = extractors.CoralReef()

coral_ads = coralreef_extractor("/home/s.kamboj.400/unzipped-coral")
# per_ads = birdset_extactor("PER")

coral_ads

Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000000.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000100.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000200.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000300.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000400.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000500.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000600.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000700.WAV
Extracting features from /home/s.kamboj.400/unzipped-coral/Degraded_Reef/April_2024_Pavonas/20240306_000

KeyboardInterrupt: 

In [8]:
coral_ads["train"].features["labels"]

Sequence(feature=ClassLabel(names=['Degraded_Reef', 'Non_Degraded_Reef'], id=None), length=-1, id=None)

In [9]:
coral_ads["train"][0]

{'sample_rate': 48000,
 'device_id': '2453AC0263FB804F',
 'duration': 60.0,
 'date': datetime.date(2024, 7, 16),
 'time': datetime.time(4, 12),
 'labels': [0, 1],
 'filepath': '/home/s.kamboj.400/unzipped-coral/Non_Degraded_Reef/July_2024/20240716_041200.WAV',
 'audio': '/home/s.kamboj.400/unzipped-coral/Non_Degraded_Reef/July_2024/20240716_041200.WAV',
 'audio_in': {'array': '/home/s.kamboj.400/unzipped-coral/Non_Degraded_Reef/July_2024/20240716_041200.WAV',
  'sampling_rate': 48000},
 'max_freq': 24000.0,
 'min_freq': 0.0,
 'mean_freq': 12840.03391277726,
 'peak_freq': 562.5,
 'variance_freq': 7647.388445669873,
 'num_peaks': 55,
 'spectral_flatness': 0.2607593238353729,
 'rms_energy': 0.04303060472011566}

In [ ]:
# import librosa

# librosa.load("data_birdset/background_noise/fff12db0-9cbe-4155-ac4a-b0b88d84c1d7.wav")

(array([ 0.19588947,  0.27712345,  0.20782189, ..., -0.04795212,
        -0.12272076, -0.17080563], shape=(220500,), dtype=float32),
 22050)

# Online Preprocessing example

Suppose we just wanted spectrograms with no audio preprocessing

In [ ]:
# hsn_ads["test"][0]["audio"]

{'bytes': None,
 'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e7a5318118cabfab47a509edeb627860a60537535aeea20a19fced4c110d579e/HSN_001_20150708_061805_000_005.ogg'}

In [ ]:
# hsn_ads["train"][0]

{'audio': {'bytes': None,
  'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg'},
 'filepath': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg',
 'start_time': None,
 'end_time': None,
 'low_freq': None,
 'high_freq': None,
 'ebird_code': 16,
 'ebird_code_multilabel': [16],
 'ebird_code_secondary': [],
 'call_type': 'call, phrases',
 'sex': None,
 'lat': 31.34,
 'long': -109.273,
 'length': 58,
 'microphone': 'focal',
 'license': '//creativecommons.org/licenses/by-nc-sa/3.0/',
 'source': 'xenocanto',
 'local_time': '08:54',
 'detected_events': [[31.44, 33.792], [35.952, 36.672]],
 'event_cluster': [-1, -1],
 'peaks': [4.41303361102967,
  5.9407844283959825,
  7.327410170258567,
  10.157911684613236,
  13.236382081634444,
  19.7893855875755,
  25.044450899102944,
  26.92607690579703,
  28.70

In [ ]:
# hsn_ads["train"][0]

{'audio': {'bytes': None,
  'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg'},
 'filepath': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg',
 'start_time': None,
 'end_time': None,
 'low_freq': None,
 'high_freq': None,
 'ebird_code': 16,
 'ebird_code_multilabel': [16],
 'ebird_code_secondary': [],
 'call_type': 'call, phrases',
 'sex': None,
 'lat': 31.34,
 'long': -109.273,
 'length': 58,
 'microphone': 'focal',
 'license': '//creativecommons.org/licenses/by-nc-sa/3.0/',
 'source': 'xenocanto',
 'local_time': '08:54',
 'detected_events': [[31.44, 33.792], [35.952, 36.672]],
 'event_cluster': [-1, -1],
 'peaks': [4.41303361102967,
  5.9407844283959825,
  7.327410170258567,
  10.157911684613236,
  13.236382081634444,
  19.7893855875755,
  25.044450899102944,
  26.92607690579703,
  28.70

In [15]:
from pyha_analyzer.preprocessors import MelSpectrogramPreprocessors

# TODO: allow for normalization system

# preprocessor acts as a function for processing
# class allows us to configure parameters and whatnot
preprocessor = MelSpectrogramPreprocessors(duration=5, class_list=coral_ads["train"].features["labels"].feature.names)

# Set split spefific transforms here
# If preprocessor had data augmentations, probably want to disable for training
coral_ads["train"].set_transform(preprocessor)
coral_ads["valid"].set_transform(preprocessor)
coral_ads["test"].set_transform(preprocessor)
coral_ads["train"][[0, 1]]["audio"][0].mean()

np.float32(0.13795403)

In [ ]:
# hsn_ads["train"][[0, 1]]

{'audio': [array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],
        shape=(1, 256, 431), dtype=float32),
  array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],
        shape=(1, 256, 431), dtype=float32)],
 'filepath': ['/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg',
  '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/053832b39de8ba5377824eb2d8364a1e31d9355ede4974f3741a1daa9822e8ab/XC700958.ogg'],
 'start_time': [None, None],
 'end_time': [None, None],
 'low_f

# Model Training

As a demo, implementing a model here because we haven't fleshed out the AudioDataset api yet

In the future this exists in `/pyha_analyzer/models/`

In [25]:
# from pyha_analyzer.models.demo_CNN import ResnetConfig, ResnetModel

# # Going to make notes on anything that should be handled not here
# # This is one of these things, this should be handled by potentially pyha_trainer
# resnet50d_config = ResnetConfig(
#     num_classes=len(hsn_ads["train"].features["ebird_code"].names), input_channels=1
# )
# model = ResnetModel(resnet50d_config)
from pyha_analyzer.models import EfficentNet
#model = EfficentNet(num_classes=len(coral_ads["train"].features["ebird_code"].names))
model = EfficentNet(num_classes=2)

In [26]:
args = PyhaTrainingArguments(
    working_dir="working_dir"
)
args.num_train_epochs = 1
args.eval_steps = 20


trainer = PyhaTrainer(
    model=model,
    dataset=coral_ads,
    training_args=args
)
trainer.train()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


train/epoch,▁
train/global_step,▁
total_flos,0
train/epoch,1
train/global_step,1
train_loss,0.68627
train_runtime,5.0105
train_samples_per_second,3.792
train_steps_per_second,0.2


Step,Training Loss,Validation Loss


TrainOutput(global_step=1, training_loss=0.6752867102622986, metrics={'train_runtime': 2.8846, 'train_samples_per_second': 6.587, 'train_steps_per_second': 0.347, 'total_flos': 0.0, 'train_loss': 0.6752867102622986, 'epoch': 1.0})

In [28]:
trainer.evaluate(eval_dataset=coral_ads["test"], metric_key_prefix="Soundscape")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f0cad9d1e50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f0cad963f10, raw_cell="trainer.evaluate(eval_dataset=coral_ads["test"], m.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.kamboj.400/pyha-analyzer-2.0/train_demo.ipynb#X22sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f0cad9d1e50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f0cadae3d90, execution_count=28 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 7f0cad963f10, raw_cell="trainer.evaluate(eval_dataset=coral_ads["test"], m.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.kamboj.400/pyha-analyzer-2.0/train_demo.ipynb#X22sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

#